In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.cuda
import torchvision.transforms.functional as TF
import torchvision.transforms as transforms
import os
from joblib import dump, load
from PIL import Image
from tqdm.notebook import tqdm
from torch.nn.functional import relu
from torch.utils.data import DataLoader, Dataset
from sklearn.manifold import TSNE
from SHG import SHG
from utils import *
from matplotlib.offsetbox import OffsetImage, AnnotationBbox
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
from sklearn.base import clone
from sklearn_extra.cluster import KMedoids

In [ ]:
def save(train_dataloader, device, model, NUM_DATAPOINTS, save = False):
    """ Finds NUM_DATAPOINTS from train_dataloader of fully annotated images """
    bottleneck_arr = []
    gt_heatmaps = []
    IDs = []

    with torch.no_grad():
        for x, y, id_ in tqdm(train_dataloader):
            x = x.to(device, dtype = torch.float)

            if (0 not in np.array(turn_featuremaps_to_keypoints(y)).reshape((-1, 3))[:, -1]):
                _, bottleneck = model(x)

                bottleneck_arr.append(bottleneck[2].cpu().data.numpy().flatten())
                gt_heatmaps.append(turn_featuremaps_to_keypoints(y))
                IDs.append(id_)

                if (len(bottleneck_arr) == NUM_DATAPOINTS):
                    break

    bottleneck_arr = np.array(bottleneck_arr).reshape((len(bottleneck_arr), -1))
    gt_heatmaps = np.array(gt_heatmaps).reshape((len(gt_heatmaps), -1))
    
    if (save):
        dump(IDs, SAVING_PATH + str(NUM_DATAPOINTS) + "/" + "IDs.joblib") # Saving the IDs of the images worked on

    return bottleneck_arr, gt_heatmaps, IDs

In [ ]:
""" PATHS """
# Input images
TRAIN_IMGS_PATH = "C:/Users/André/OneDrive 2/OneDrive/Skrivebord/bsc_data/train/image/"

# Ground truth heatmaps
TRAIN_HEATMAPS_PATH = "C:/Users/André/OneDrive 2/OneDrive/Skrivebord/bsc_data/train/heatmaps/"

# Path for model
MODEL_DIR = "C:/Users/André/OneDrive 2/OneDrive/Skrivebord/bsc_data/models/Mon_Apr_12_18-11-59_2021/epoch_46.pth"

# Path for mean rgb
average_rgb = np.loadtxt("./average_rgb.npy")

# Saving path
SAVING_PATH = "C:/Users/André/OneDrive 2/OneDrive/Skrivebord/bsc_data/latent_space_data/data/"

In [ ]:
""" DATASET AND DATALOADER """

class dataset(Dataset):
    def __init__(self, X_path, y_path, average_rgb):
        self.X_path = X_path
        self.y_path = y_path
        self.X_data = os.listdir(self.X_path)
        self.average_rgb = average_rgb
        self.norm = transforms.Normalize(mean = self.average_rgb, std = [1, 1, 1])

    def __len__(self):
        return len(self.X_data)

    def __getitem__(self, i):
        ID = self.X_data[i]
        x = Image.open(self.X_path + ID)

        y = []

        for i in range(17):
            y.append(torch.from_numpy(np.load(self.y_path + ID[:-4] + "/" + str(i) + ".npy")))

        x = TF.to_tensor(x)

        if (x.shape[0] == 1): # If the image is gray-scale, cast it to rgb
            x = torch.stack((x[0],) * 3)

        x = self.norm(x) # Subtracts mean rgb

        y = torch.stack(y)
        return x, y, ID

train_data = dataset(TRAIN_IMGS_PATH, TRAIN_HEATMAPS_PATH, average_rgb)
train_dataloader = DataLoader(train_data, batch_size = 1, shuffle = True)

In [ ]:
""" LOADING MODEL """
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = SHG(num_hourglasses = 1, use_skip_connections = False).to(device)
model.load_state_dict(torch.load(MODEL_DIR))
model.eval()
print()

In [ ]:
 """ Save each fully annotated image, from the bottleneck """
 
 with torch.no_grad():
    for x, y, id_ in tqdm(train_dataloader):
        x = x.to(device, dtype = torch.float)

        if (0 not in np.array(turn_featuremaps_to_keypoints(y)).reshape((-1, 3))[:, -1]):
            _, bottleneck = model(x)

            bottleneck = bottleneck[-1][0].cpu().data.numpy()
            np.save(SAVING_PATH + id_[0][:-4], bottleneck)